# Work FLow
    1.import Libraries
    2.input embedding
    3.Positional Encoding
    4.Multi Head Attention(Self Attention)
    5.Add & Normalized
    6.Feed Forward
    7.Residual Connection
    8.Encoder
    9.Decoder
    10.Build a Transformer
    11.Testing Transformer
    12.Tokenizer
    13.Loading Dataset
    14.Valid Loop
    15.Traning Loop
    16.Conclusion

## 1.import Libraries

In [10]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset,random_split
from torch.utils.tensorboard import SummaryWriter

import math

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

from pathlib import Path

from typing import Any

from tqdm import tqdm

# 2.Input Embedding

In [11]:
class InputEmbedding(nn.Module):
    def __init__(self,d_model:int,vocab_size:int):
        super().__init__()
        self.d_model=d_model
        self.vocab_size = vocab_size
        self.embedding=nn.Embedding(vocab_size,d_model)
        
    def forward(self,x):
        return self.embedding(x)*math.sqrt(self.d_model)

# 3.Position Encoding 

In [14]:
class PositionEncoding(nn.Module): 
    def __init__(self, d_model: int, seq_len: int, dropout: float):
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        
        pe = torch.zeros(seq_len, d_model)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)  # (seq_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # (1, seq_len, d_model)
        
        self.register_buffer("pe", pe)
        
    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

## 4.Multi-Head Attention

In [15]:
class MultiHeadAttention(nn.Module):
    def __init__(self,d_model:int,h:int,dropout:float):
        super().__init__()
        self.d_model=d_model
        self.h=h
        
        assert d_model%h==0,"d_model is not divisible by h"
        
        self.d_k=d_model//h
        
        self.w_q=nn.Linear(d_model,d_model)
        self.w_k=nn.Linear(d_model,d_model)
        self.w_v=nn.Linear(d_model,d_model)
        self.w_o=nn.Linear(d_model,d_model)
        
        self.dropout=nn.Dropout(dropout)
        
    @staticmethod
    def attention(query,key,value,mask,dropout:nn.Dropout):
        d_k=query.shape[-1]
        attention_scores=(query @ key.transpose(-2,-1))/math.sqrt(d_k)
        
        if mask is not None:
            attention_scores.masked_fill(mask==0,-1e9)
        attention_scores=attention_scores.softmax(dim=-1)
        
        if dropout is not None:
            attention_scores=dropout(attention_scores)
            
        return (attention_scores@value),attention_scores
            
    def forward(self,q,k,v,mask):
        query=self.w_q(q)
        key=self.w_k(k)
        value=self.w_v(v)
        
        query=query.view(query.shape[0],query.shape[1],self.h,self.d_k).transpose(1,2)
        key=key.view(key.shape[0],key.shape[1],self.h,self.d_k).transpose(1,2)
        value=value.view(value.shape[0],value.shape[1],self.h,self.d_k).transpose(1,2)
        
        x,self.attention_scores=MultiHeadAttention.attention(query,key,value,mask,self.dropout)
        
        x=x.transpose(1,2).contiguous().view(x.shape[0],-1,self.h*self.d_k)
        
        return self.w_o(x)